# ガウスルジャンドル法で積分値を計算してみよう

$[-1,1]$に標準化された区間の積分値を求める方法．シンプソン法に比べ，少ない計算コストで計算を実行することができ，(特に予め後述の重み係数$w_i$を準備しておけば)自然科学における数値計算で有用であることが期待されている．

\begin{eqnarray}
    I = \int_{-1}^{1} f(x)dx = \sum_{i=1}^{n} w_i f(x_i)
\end{eqnarray}

ただし，
\begin{eqnarray}
    w_i = \frac{2}{(1-x_i^2)\left[P_n'(x_i)\right]^2}
\end{eqnarray}
で，$P_n(x)$はルジャンドル関数，$P_n'(x)$はその微分，$x_i$はノード($P(x_i)=0$となるような点)である．

$w_i$は重み係数と呼ばれる．
次数が$2n-1$以下の多項式の積分値を厳密に求めることができる．

> 一般に積分範囲は任意に取ることができるが，平行移動や変数変換を行うことにより，積分区間が$[-1,1]$になるように規格化することができる．

ノードを(三角行列の固有値問題に落とし込み)求めるアルゴリズムが知られているが，本ファイルではそれは実装しない．また，積分区間を$[-1,1]$ではなく任意に取ることができるが，その実装もここでは行わない．

## 積分の計算
\begin{eqnarray}
    I = \int_{-1}^{1} \frac{2}{1+x^2} dx = \pi
\end{eqnarray}

## ルジャンドル関数の漸化式
$P_n(1)=1$で規格化されているとすると
\begin{eqnarray}
    P_n(x) =&& \frac{2n-1}{n}x P_{n-1} - \frac{n-1}{n}P_{n-2}(x)\\
    && P_0(x) = 1, ~~ P_1(x) = x
\end{eqnarray}

In [1]:
import numpy as np
from scipy.special import eval_legendre
from scipy.special import legendre
from fractions import Fraction

In [2]:
class poly:
    def __init__(self,poly_array):
        if type(poly_array)!=(np.ndarray or list):
            poly_list = [poly_array]
            self.array = np.array(poly_list)
            self.list = poly_list
            self.len = 1
        else:
            self.array = poly_array
            self.list = list(poly_array)
            self.len = len(poly_array)
    def __add__(self,other):
        """
        fixed numpy add method so that can be applied to different array length addition
        """
        if self.len>other.len:
            for i in range(abs(self.len-other.len)):
                other.list.append(0)
            other.array = np.array(other.list)
        elif self.len<other.len:
            for i in range(abs(self.len-other.len)):
                self.list.append(0)
            self.array = np.array(self.list)
        return self.array + other.array
    def __sub__(self,other):
        """
        fixed numpy add method so that can be applied to different array length addition
        """
        if self.len>other.len:
            for i in range(abs(self.len-other.len)):
                other.list.append(0)
            other.array = np.array(other.list)
        elif self.len<other.len:
            for i in range(abs(self.len-other.len)):
                self.list.append(0)
            self.array = np.array(self.list)
        return self.array - other.array
    def __mul__(self,other):
        mul_poly_array = np.array([0.0]*(self.len+other.len-1))
        for i in range(self.len):
            for j in range(other.len):
                mul_poly_array[i+j]+=self.array[i]*other.array[j]
        return mul_poly_array
    def __str__(self):
        poly_str = f"{self.array[0]}"
        for i in range(1,self.len):
            poly_str += f" + {self.array[i]} x^{i}"
        return poly_str
    def root(array):
        """
        returns roots of "poly = 0".
        """
        return np.roots(array[::-1])
    def val(array,x):
        """
        returns value of x
        """
        return sum(array[i]*x**i for i in range(len(array)))
    def differential(array,order):
        dif_list = []
        for i in range(order,len(array)):
            coeff = 1
            for j in range(order):
                coeff*=(i-j)
            dif_list.append(coeff*array[i])
        return np.array(dif_list)
    
def array_all_legendre(n):
    """
    normalized as P(1)=1
    get legendre funcion up to whose order is n
    """
    all_legendre = [np.array([1]),np.array([0,1])]
    if n>=2:
        for i in range(2,n+1):
            first_poly = poly(np.array([0,(2*i-1)/i])) * poly(all_legendre[-1])
            second_poly = poly(np.array([(i-1)/i])) * poly(all_legendre[-2])
            all_legendre.append(poly(first_poly) - poly(second_poly))
        return np.array(all_legendre)
    elif n==1:
        return np.array(all_legendre)
    else:
        return all_legendre[0]

class Legendre:
    def __init__(self,n):
        self.order = n
    def get_array(n):
        """
        normalized as P(1)=1
        get legendre funcion up to whose order is n
        """
        all_legendre = [np.array([1]),np.array([0,1])]
        if n>=2:
            for i in range(2,n+1):
                first_poly = poly(np.array([0,(2*i-1)/i])) * poly(all_legendre[-1])
                second_poly = poly(np.array([(i-1)/i])) * poly(all_legendre[-2])
                all_legendre.append(poly(first_poly) - poly(second_poly))
            return np.array(all_legendre)[-1]
        elif n==1:
            return np.array(all_legendre)[-1]
        else:
            return all_legendre[0]
    def find_nodes(n):
        return np.roots(array_all_legendre(n)[-1][::-1])


def weighted_coeff(n_nodes,array):
    dif_leg = poly.val(poly.differential(Legendre.get_array(n_nodes),1),array)
    return 2/((1-array**2)*dif_leg**2)

def gauss_legendre(n_nodes,func):
    area = 0
    nodes = Legendre.find_nodes(n_nodes)
    for i in range(n_nodes):
        area+=weighted_coeff(n_nodes,nodes[i])*func(nodes[i])
        #print(weighted_coeff(n_nodes,nodes[i]),func(nodes[i]))
    return area

In [3]:
n_nodes = 5

def lorentz(x):
    return 2/(1+x**2)

def parabola(x):
    """
    x**2+x+1 = (x+1/2)**2+3/4
    """
    return x**2+x+1

print(gauss_legendre(n_nodes=n_nodes,func=lorentz))
print(gauss_legendre(n_nodes=n_nodes,func=parabola))

3.142342342342345
2.6666666666666714


/Users/masahikokamoshita/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/masahikokamoshita/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## シンプソン法との比較

In [4]:
def simpson_integral(array,mode:int,x1:float,x2:float)->float:
    dx = (x2-x1)/(len(array)-1)
    area = 0
    if mode==0:
        area = np.sum(array)*dx
    elif mode==1:
        for i in range(len(array)-1):
            area += (array[i]+array[i+1]) *dx/2
    elif mode==2:
        if len(array)>=3:
            list_coeff = [3-(-1)**i for i in range(len(array))]
            if len(array)%2==1:
                list_coeff[0],list_coeff[-1] = 1,1
            else:
                list_coeff[0],list_coeff[-2],list_coeff[-1] = 1,2.5,1.5
        elif len(array)==2:
            list_coeff = [1.5,1.5]
        elif len(array)==1:
            list_coeff = [3]
        else:
            raise(NotImplementedError)
        array_coeff = np.array(list_coeff)
        area = np.sum(array_coeff*array)*dx*2/6
    elif mode==3:
        unit = [2,3,3]
        if len(array)>=4:
            list_coeff = unit*((len(array)-1)//3)
            list_coeff[0]=1
            if len(array)%3==1:
                list_coeff+=[1]
            elif len(array)%3==2:
                list_coeff+=[7/3,4/3]
            else:
                list_coeff+=[17/9,32/9,8/9]
        elif len(array)==1:
            list_coeff = [8/3]
        elif len(array)==2:
            list_coeff = [4/3,4/3]
        elif len(array)==3:
            list_coeff = [8/9,32/9,8/9]
        else:
            raise(NotImplementedError)
        array_coeff = np.array(list_coeff)
        area = np.sum(array_coeff*array)*dx*3/8
    else:
        raise(NotImplementedError)
    return area

x1,x2=-1,1
x = np.linspace(x1,x2,5)
array_lorentz = lorentz(x)
array_parabola = parabola(x)

print(simpson_integral(array_parabola,mode=2,x1=x1,x2=x2))
print(simpson_integral(array_lorentz,mode=2,x1=x1,x2=x2))

2.6666666666666665
3.1333333333333333
